
# Descomprimir MetStation y reunir archivos `.25O` por estación


In [1]:
# === CONFIGURACIÓN DE RUTAS (MANCHI) ===
from pathlib import Path

# Carpeta con los .zip de MANCHI
ZIP_ROOT = Path(r"C:\Users\varga\Box\Data\RinexZipFiles\MANCHI\Last")

# Carpeta base donde se crearán subcarpetas por estación
DEST_ROOT = Path(r"C:\Users\varga\Box\Data\nc Files\MANCHI")

# Subcadena para identificar la carpeta RINEX
RINEX_SUBSTR = "RINEX_3_03"

# Patrón de zip a considerar
ZIP_PATTERN = "MANCHI*.zip"

# Carpeta donde se extraerán los zips
UNZIP_ROOT = ZIP_ROOT / "unzipped"

# Política de sobre-escritura de .25O
OVERWRITE_25O = False

ZIP_ROOT, DEST_ROOT, UNZIP_ROOT


(WindowsPath('C:/Users/varga/Box/Data/RinexZipFiles/MANCHI/Last'),
 WindowsPath('C:/Users/varga/Box/Data/nc Files/MANCHI'),
 WindowsPath('C:/Users/varga/Box/Data/RinexZipFiles/MANCHI/Last/unzipped'))

In [2]:
# === LÓGICA: extraer zips y copiar TODOS los *.25O a una sola carpeta (MANCHI) ===
import zipfile, shutil

def safe_extract(zf: zipfile.ZipFile, dest: Path):
    for member in zf.infolist():
        member_path = dest / member.filename
        if not str(member_path.resolve()).startswith(str(dest.resolve())):
            raise RuntimeError(f"Entrada peligrosa en ZIP: {member.filename}")
        zf.extract(member, dest)

def extract_all_zips(root: Path, unzip_root: Path, pattern: str):
    unzip_root.mkdir(parents=True, exist_ok=True)
    zips = sorted(root.glob(pattern), key=lambda p: p.name.lower())
    extracted = []
    for z in zips:
        target_dir = unzip_root / z.stem
        if target_dir.exists():
            extracted.append(target_dir)
            continue
        try:
            with zipfile.ZipFile(z) as zf:
                print(f"[EXTRACT] {z.name} -> {target_dir.name}")
                target_dir.mkdir(parents=True, exist_ok=True)
                safe_extract(zf, target_dir)
                extracted.append(target_dir)
        except Exception as e:
            print(f"[ERROR] {z.name}: {e}")
    return extracted

def find_rinex_dirs(base: Path, substr: str):
    sub = substr.lower()
    return [p for p in base.rglob("*") if p.is_dir() and sub in p.name.lower()]

def copy_25O_flat(unzip_root: Path, dest_root: Path, rinex_substr: str, overwrite=False):
    dest_root.mkdir(parents=True, exist_ok=True)
    total = 0
    for station_dir in sorted(p for p in unzip_root.iterdir() if p.is_dir()):
        rinex_dirs = find_rinex_dirs(station_dir, rinex_substr)
        if rinex_dirs:
            for rdir in rinex_dirs:
                for f in rdir.glob("*.25O"):
                    df = dest_root / f.name
                    if df.exists() and not overwrite:
                        continue
                    shutil.copy2(f, df)
                    total += 1
        else:
            for f in station_dir.rglob("*.25O"):
                df = dest_root / f.name
                if df.exists() and not overwrite:
                    continue
                shutil.copy2(f, df)
                total += 1
    return total

# === Ejecutar flujo ===
DEST_ROOT.mkdir(parents=True, exist_ok=True)
extracted_dirs = extract_all_zips(ZIP_ROOT, UNZIP_ROOT, ZIP_PATTERN)
for p in sorted(UNZIP_ROOT.iterdir() if UNZIP_ROOT.exists() else []):
    if p.is_dir() and p not in extracted_dirs:
        extracted_dirs.append(p)

total_copiados = copy_25O_flat(UNZIP_ROOT, DEST_ROOT, RINEX_SUBSTR, overwrite=OVERWRITE_25O)
print("\n===== RESUMEN (Notebook) =====")
print(f"Zips procesados/existentes: {len(extracted_dirs)}")
print(f"Archivos *.25O copiados en total: {total_copiados}")
print(f"Destino único: {DEST_ROOT}")



===== RESUMEN (Notebook) =====
Zips procesados/existentes: 116
Archivos *.25O copiados en total: 0
Destino único: C:\Users\varga\Box\Data\nc Files\MANCHI


## Libraries

In [3]:
%load_ext autoreload
%autoreload 2
import gnssvod as gv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pdb
import importlib
import zipfile
import os
import xarray as xr
import glob
import shutil
from gnssvod.hemistats.hemistats import hemibuild
#import georinex as gr
#import qgrid as interactive table 
from matplotlib.collections import PatchCollection
import matplotlib.dates as mdates
import shutil
from matplotlib import rcParams
from datetime import datetime 
import shutil
from pathlib import Path


## Presets

### Time intervals

In [4]:
# Define the start day
startday = pd.to_datetime('25-04-2025', format='%d-%m-%Y')
# Generate a range of datetime values
timeintervals=pd.interval_range(start=startday, periods=10, freq='D', closed='left')
timeintervals

IntervalIndex([[2025-04-25 00:00:00, 2025-04-26 00:00:00),
               [2025-04-26 00:00:00, 2025-04-27 00:00:00),
               [2025-04-27 00:00:00, 2025-04-28 00:00:00),
               [2025-04-28 00:00:00, 2025-04-29 00:00:00),
               [2025-04-29 00:00:00, 2025-04-30 00:00:00),
               [2025-04-30 00:00:00, 2025-05-01 00:00:00),
               [2025-05-01 00:00:00, 2025-05-02 00:00:00),
               [2025-05-02 00:00:00, 2025-05-03 00:00:00),
               [2025-05-03 00:00:00, 2025-05-04 00:00:00),
               [2025-05-04 00:00:00, 2025-05-05 00:00:00)],
              dtype='interval[datetime64[ns], left]')

## Process Rinex Files

In [5]:
from pathlib import Path

# Carpeta única donde se copiaron todos los .25O de MANCHI
root = Path(r"C:\Users\varga\Box\Data\nc Files\MANCHI")

# Buscar todos los archivos con extensión .25O
files = list(root.glob("*.25O"))

print("Found", len(files), "files:")
for f in files:
    print(f.name)


Found 116 files:
MANCHI_raw_20250413142051.25O
MANCHI_raw_20250414142051.25O
MANCHI_raw_20250415142052.25O
MANCHI_raw_20250415223240.25O
MANCHI_raw_20250416223240.25O
MANCHI_raw_20250417223241.25O
MANCHI_raw_20250418223242.25O
MANCHI_raw_20250419223244.25O
MANCHI_raw_20250420223245.25O
MANCHI_raw_20250421223246.25O
MANCHI_raw_20250422223247.25O
MANCHI_raw_20250423223248.25O
MANCHI_raw_20250424223249.25O
MANCHI_raw_20250425223250.25O
MANCHI_raw_20250426223251.25O
MANCHI_raw_20250427223252.25O
MANCHI_raw_20250428223252.25O
MANCHI_raw_20250429223253.25O
MANCHI_raw_20250502142647.25O
MANCHI_raw_20250503142648.25O
MANCHI_raw_20250504142649.25O
MANCHI_raw_20250505142649.25O
MANCHI_raw_20250506142650.25O
MANCHI_raw_20250507142651.25O
MANCHI_raw_20250508142652.25O
MANCHI_raw_20250509142653.25O
MANCHI_raw_20250510142653.25O
MANCHI_raw_20250511142654.25O
MANCHI_raw_20250512142655.25O
MANCHI_raw_20250513142656.25O
MANCHI_raw_20250514142657.25O
MANCHI_raw_20250515142658.25O
MANCHI_raw_202505161426

In [6]:
from pathlib import Path
import glob
import xarray as xr
import gnssvod as gv

# Carpeta donde dejaste TODOS los .25O de MANCHI
DEST = Path(r"C:\Users\varga\Box\Data\nc Files\MANCHI")

# 1) Lista de archivos de entrada (RINEX .25O)
rinex_files = sorted(glob.glob(str(DEST / '*.25O')))
print(f'Found {len(rinex_files)} .25O files (showing up to 5):', rinex_files[:5])
if len(rinex_files) == 0:
    raise FileNotFoundError("No .25O files found in DEST folder. Check DEST and file existence.")

# 2) Carpeta de salida para NetCDF (dentro de MANCHI)
outdir = DEST / "processed"
outdir.mkdir(parents=True, exist_ok=True)

# 3) Carpeta persistente para órbitas/relojes
aux_dir = DEST / "aux_orbits"
aux_dir.mkdir(parents=True, exist_ok=True)

# 4) Variables a conservar
keepvars = ['S?', 'S??']

ok, bad = [], []

# 5) Procesar uno por uno
for rinex in rinex_files:
    try:
        pattern = {'MANCHI': rinex}
        outputdir = {'MANCHI': str(outdir)}

        result = gv.preprocess(
            pattern,
            interval='15s',
            keepvars=keepvars,
            outputdir=outputdir,
            outputresult=True,
            aux_path=str(aux_dir)
        )

        manchi_res = result.get('MANCHI', [])
        if isinstance(manchi_res, list):
            if not manchi_res:
                raise ValueError(f"No dataset produced for: {rinex}")
            _ = manchi_res[0]
        elif isinstance(manchi_res, xr.Dataset):
            _ = manchi_res
        else:
            raise TypeError(f"Unexpected return type for 'MANCHI': {type(manchi_res)}")

        print(f"[OK] {Path(rinex).name}")
        ok.append(rinex)

    except Exception as e:
        print(f"[WARN] Skipping due to error: {Path(rinex).name}")
        print("       ", e)
        bad.append((rinex, str(e)))
        continue

# 6) Resumen final
print("\n===== SUMMARY =====")
print(f"Processed OK: {len(ok)}")
print(f"Failed     : {len(bad)}")

if bad:
    print("\nFiles that failed:")
    for fname, err in bad:
        print(f" - {fname}: {err}")
else:
    print("\n¡Todo fue un éxito! 🤙")


Found 116 .25O files (showing up to 5): ['C:\\Users\\varga\\Box\\Data\\nc Files\\MANCHI\\MANCHI_raw_20250413142051.25O', 'C:\\Users\\varga\\Box\\Data\\nc Files\\MANCHI\\MANCHI_raw_20250414142051.25O', 'C:\\Users\\varga\\Box\\Data\\nc Files\\MANCHI\\MANCHI_raw_20250415142052.25O', 'C:\\Users\\varga\\Box\\Data\\nc Files\\MANCHI\\MANCHI_raw_20250415223240.25O', 'C:\\Users\\varga\\Box\\Data\\nc Files\\MANCHI\\MANCHI_raw_20250416223240.25O']
Could not find any files matching the pattern C:\Users\varga\Box\Data\nc Files\MANCHI\processed*.nc
C:\Users\varga\Box\Data\nc Files\MANCHI\MANCHI_raw_20250413142051.25O exists | Reading...
Processed 0 out of 2919668 lines (0.0%)
Processed 738598 out of 2919668 lines (25.3%)
Processed 1271476 out of 2919668 lines (43.5%)
Processed 1893488 out of 2919668 lines (64.9%)
Processed 2761302 out of 2919668 lines (94.6%)
Observation file  C:\Users\varga\Box\Data\nc Files\MANCHI\MANCHI_raw_20250413142051.25O  is read in 315.47 seconds.
Processing 2833268 individ

GFZ0MGXRAP_20251050000_01D_05M_ORB.SP3.gz: 1.04MB [00:03, 313kB/s]                             


 | Download completed for ftp://gssc.esa.int/gnss/products/2362//GFZ0MGXRAP_20251050000_01D_05M_ORB.SP3.gz
C:\Users\varga\Box\Data\nc Files\MANCHI\aux_orbits\GFZ0MGXRAP_20251050000_01D_05M_ORB.SP3 file is read in 8.74 seconds
This file does not exist: C:\Users\varga\Box\Data\nc Files\MANCHI\aux_orbits\GFZ0MGXRAP_20251060000_01D_05M_ORB.SP3
Downloading: ftp://gssc.esa.int/gnss/products/2362//GFZ0MGXRAP_20251060000_01D_05M_ORB.SP3.gz

GFZ0MGXRAP_20251060000_01D_05M_ORB.SP3.gz: 1.03MB [00:05, 180kB/s]                             


 | Download completed for ftp://gssc.esa.int/gnss/products/2362//GFZ0MGXRAP_20251060000_01D_05M_ORB.SP3.gz
C:\Users\varga\Box\Data\nc Files\MANCHI\aux_orbits\GFZ0MGXRAP_20251060000_01D_05M_ORB.SP3 file is read in 7.70 seconds
This file does not exist: C:\Users\varga\Box\Data\nc Files\MANCHI\aux_orbits\GFZ0MGXRAP_20251050000_01D_30S_CLK.CLK
Downloading: GFZ0MGXRAP_20251050000_01D_30S_CLK.CLK.gz

...

 | Download completed for GFZ0MGXRAP_20252340000_01D_30S_CLK.CLK.gz
C:\Users\varga\Box\Data\nc Files\MANCHI\aux_orbits\GFZ0MGXRAP_20252340000_01D_30S_CLK.CLK file is read in 2.01 seconds
SP3 interpolation is done in 3.23 seconds
Saved 89514 individual observations in C:\Users\varga\Box\Data\nc Files\MANCHI\processed\MANCHI_raw_20250821133358.nc
[OK] MANCHI_raw_20250821133358.25O
Could not find any files matching the pattern C:\Users\varga\Box\Data\nc Files\MANCHI\processed*.nc
C:\Users\varga\Box\Data\nc Files\MANCHI\MANCHI_raw_20250822133359.25O exists | Reading...
Observation file  C:\Users\varga\Box\Data\nc Files\MANCHI\MANCHI_raw_20250822133359.25O  is read in 2.96 seconds.
Processing 85948 individual observations
Calculating Azimuth and Elevation
C:\Users\varga\Box\Data\nc Files\MANCHI\aux_orbits\GFZ0MGXRAP_20252340000_01D_05M_ORB.SP3 exists | Reading...
C:\Users\varga\Box\Data\nc Files\MANCHI\aux_orbits\GFZ0MGXRAP_20252340000_01D_05M_ORB.SP3 file is read in 0.68 seconds
This file 

GFZ0MGXRAP_20252350000_01D_05M_ORB.SP3.gz: 0.00B [00:02, ?B/s]


<urlopen error <urlopen error ftp error: error_perm('550 CWD command failed: directory not found.')>>
 | Requested file ftp://gssc.esa.int/gnss/products/2380//GFZ0MGXRAP_20252350000_01D_05M_ORB.SP3.gz cannot be not found!
[WARN] Skipping due to error: MANCHI_raw_20250822133359.25O
        [Errno 2] No such file or directory: 'C:\\Users\\varga\\Box\\Data\\nc Files\\MANCHI\\aux_orbits\\GFZ0MGXRAP_20252350000_01D_05M_ORB.SP3'
Could not find any files matching the pattern C:\Users\varga\Box\Data\nc Files\MANCHI\processed*.nc
C:\Users\varga\Box\Data\nc Files\MANCHI\MANCHI_raw_20250823133400.25O exists | Reading...
Observation file  C:\Users\varga\Box\Data\nc Files\MANCHI\MANCHI_raw_20250823133400.25O  is read in 1.19 seconds.
Processing 50542 individual observations
Calculating Azimuth and Elevation
This file does not exist: C:\Users\varga\Box\Data\nc Files\MANCHI\aux_orbits\GFZ0MGXRAP_20252350000_01D_05M_ORB.SP3
Downloading: ftp://gssc.esa.int/gnss/products/2380//GFZ0MGXRAP_20252350000_01D

GFZ0MGXRAP_20252350000_01D_05M_ORB.SP3.gz: 0.00B [00:02, ?B/s]

<urlopen error <urlopen error ftp error: error_perm('550 CWD command failed: directory not found.')>>
 | Requested file ftp://gssc.esa.int/gnss/products/2380//GFZ0MGXRAP_20252350000_01D_05M_ORB.SP3.gz cannot be not found!
[WARN] Skipping due to error: MANCHI_raw_20250823133400.25O
        [Errno 2] No such file or directory: 'C:\\Users\\varga\\Box\\Data\\nc Files\\MANCHI\\aux_orbits\\GFZ0MGXRAP_20252350000_01D_05M_ORB.SP3'

===== SUMMARY =====
Processed OK: 112
Failed     : 4

Files that failed:
 - C:\Users\varga\Box\Data\nc Files\MANCHI\MANCHI_raw_20250413142051.25O: Compressed file ended before the end-of-stream marker was reached
 - C:\Users\varga\Box\Data\nc Files\MANCHI\MANCHI_raw_20250414142051.25O: Compressed file ended before the end-of-stream marker was reached
 - C:\Users\varga\Box\Data\nc Files\MANCHI\MANCHI_raw_20250822133359.25O: [Errno 2] No such file or directory: 'C:\\Users\\varga\\Box\\Data\\nc Files\\MANCHI\\aux_orbits\\GFZ0MGXRAP_20252350000_01D_05M_ORB.SP3'
 - C:\Us